# Galaxies and the large-scale structure of the Universe, Part II

_A continuation of python exercise notebook written by Rita Tojeiro, October 2017, for the Lab component of the AS1001 Cosmology module at the University of St Andrews, UK.  It was modified to stand alone in an SDSS EPO workshop at AAS in Jan 2018 and then significantly modified to emphasize programming skills rather than Astronomy data analysis by Andrea Lommen, Haverford College, January 2018. 
This notebook has benefited from examples provided by Britt Lundgren (University of North Carolina) and Jordan Raddick (John Hopkins University)_.

In this Notebook, you will use data from the Sloan Digital Sky Survey (SDSS), to explore the relationship between galaxy properties and the large-scale structure of the Universe. 

In the end, you should have found an answer to the following questions:

- How are galaxies spatially distributed in the Universe?
- Are galaxies all the same colour?
- Are galaxies all the same shape?
- How are galaxies' colours and shapes related to their spacial distribution?

In the previous lab you mastered:
- if statements, Boolean operators, arithmetic operations, and mean/min as necessary.
- the use of np.where.
- Writing variables into strings.
And you also used a SQL query to read in data. ('SQL' is pronounced just like "sequel" by the way.)

In this lab you will master:

- visualizing data by making plots and taking various "slices"
- combined data structures (lists of lists), in this case it's the use of data frames in a library called "pandas". You actually used it last lab but didn't take advantage of it. I think of it like a table of data with rows and columns rather than a list of lists.

The subjects you need to tackle in pre-class lectures in class discussions are:

- How does looking at redshift "slices" help you study the universe?
- Why are some galaxies bluer/redder than others?
- Panda dataframes, and selecting subsections of the data using indexing and np.where


### Imports

Again, we import the necessary SciServer and support libraries. 

In [ ]:
# Import Python libraries to work with SciServer
import SciServer.CasJobs as CasJobs # query with CasJobs
import SciServer.SciDrive as SciDrive   # read/write to/from SciDrive
import SciServer.SkyServer as SkyServer   # show individual objects and generate thumbnail images through SkyServer
print('SciServer libraries imported')

# Import other libraries for use in this notebook.
import numpy as np                  # standard Python lib for math ops
from scipy.misc import imsave       # save images as files
import pandas as pd                     # data manipulation package
import matplotlib.pyplot as plt     # another graphing package
import os                           # manage local files in your Compute containers
print('Supporting libraries imported')

import astroML
from astroML.datasets import fetch_sdss_spectrum
from astropy.io import ascii

# Apply some special settings to the imported libraries
# ensure columns get written completely in notebook
pd.set_option('display.max_colwidth', -1)
# do *not* show python warnings 
import warnings
warnings.filterwarnings('ignore')
print('Settings applied')

## Now that you've done some work with the output of a simple SQL query, we'll add some complexity
When we performed the previous queries (in lab2) we SELECTed only one keyword (e.g. RA, dec, redshift) at a time. That wasn't actually using the query to its full potential or using python to its full potential. We can actually ask for a number of different variables at a time. You'll notice the "SELECT" option below asks for a dozen different things. To add to the complexity, not all of them are in the "galaxy" database, so we have to "JOIN" the galaxy database with another database called "SpecObj". If you're interested in JOINs I can give you more information, but it's not something I expect you to deal with.

In [ ]:
# Find objects in the Sloan Digital Sky Survey's Data Release 14.
#
# Query the Sloan Digital Sky Serveys' Data Release 14.
# For the database schema and documentation see http://skyserver.sdss.org/dr14
#
# This query finds all galaxies with a size (petror90_r) greater than 10 arcseconds, within
# a region of sky with 100 < RA < 250, a redshift between 0.02 and 0.5, and a g-band magnitude brighter than 17.
# 
# First, store the query in an object called "query"
query="""
SELECT p.objId,p.ra,p.dec,p.petror90_r, p.expAB_r,
    p.dered_u as u, p.dered_g as g, p.dered_r as r, p.dered_i as i, 
    s.z, s.plate, s.mjd, s.fiberid
FROM galaxy AS p
   JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE p.petror90_r > 10
  and p.ra between 100 and 250
  and s.z between 0.02 and 0.5
  and p.g < 17
"""
#Then, query the database. The answer is a table that is being returned to a dataframe that we've named all_gals.
all_gals = CasJobs.executeQuery(query, "dr14")

print("SQL query finished.")
print("SQL query returned " + str(len(all_gals))+ " galaxies")

First, let's see what we ended up with. Execute the following command to see the first 10 rows of the table.

In [ ]:
all_gals[0:10]

In [ ]:
#What's the difference between that (above) and this:
all_gals.head()

Answer (1 point): 

What you just saw above: a variable name 'all_gals' then a period then an attribute 'head()' is a pattern you will see a lot. 

Now that you've seen what the output of our fancy SQL query was, look back and match up what you got with what was in the SQL query.  Describe how the SELECT part of the query determined what the columns of the table would be. (Note that you may have to use the scroll bar to see all the columns.)

Answer (1 point): 

The dataframe that is returned, which we named all_gals, holds the following quantities (in columns) for each galaxy:

- ra = Right Ascencion coordinate in degrees
- dec = Declination coordinate in degrees
- petror90_r = Radius enclosing 90% of the pertrosian flux in arcseconds. I.e., size of the galaxy in the sky.
- dered_u, dered_g, dered_r, dered_i, dered_z = Magnitudes in 5 optical filters, from the blue to the red, after subtracting the attenuation due to the Milky Way.
- z = Redshift of the galaxy
- plate = Plate number (SDSS used alluminium plates with drilled holes for positioning optical fibers).
- mjd = Date of the observation
- fiberid = Number of the fiber in a given plate. Plates have between 640 and 1000 fibers.

Let's practice manipulating this table.  If I only wanted to look at the columns u,g, and r there are two good ways to do this. (Notice in the first case that I can look at the columns in any order I want to. This is one of the things people love about these complex data structures. They do indeed present an extra layer of complexity, but you get some significant benefits, and this is one of them.)

In [ ]:
all_gals[["r","g","u"]][0:10]

In [ ]:
all_gals.loc[0:9,'u':'r']  #This is called 'slicing' a table
# Here you are using loc[] to ask for a subset of all_gals. You 
# asked for the rows from 0-9 and the columns from "u" to "r".
# Also notice that "loc" is inclusive of the endpoints (it includes
# row labeled by 9, whereas the previous command doesn't include
# the endpoint')

Your turn!  Show only the first 7 rows and only the columns from "i" to "fiberid".

In [ ]:
#Answer: (2 points)

A lot of your work in this lab will be done on an entire column.
There are various ways to select a column to work with. Here are three examples using the "ra" column.
* all_gals["ra"] 
* all_gals.loc[ : ,"ra"]
* all_gals.loc["ra"]

The ":" without anything on either side of it tells it to just use all the rows.

What you've been doing here (besides practicing manipulating arrays) is getting to know the data you've read in. Anytime you read in a bunch of data it's a really good idea to get a sense of what the data look like, and whether it looks reasonable. For example, are the columns really as described above?  Or for example did the plate numbers actually get put in the MJD column, because we made some mistake reading it in? We should confirm the data was read in correctly before doing fancy things with the data (which we will eventually do!) 



## The large scale structure of the Universe 

### Exercise:

1. Plot the positions of all galaxies usint plt.scatter(). Remember to add labels and a title to your plot. Given the large number of points, you might want to use marker='.' and s=1 (those are options in plt.scatter - you just put them inside the parenthesis separated with commas.

2. What can you tell from the distribution of galaxies? Are they uniformly distributed on the sky?

In [ ]:
#Answer (4 points)

Answer: (1 point)

### Exercise: 

Using the np.where() command, select galaxies in two narrow redshift slices:
- slice 1: 0.02 < z < 0.03 (green)
- slice 2: 0.03 < z < 0.04 (orange)

Make the same plot as above, but only using the galaxies in each slice using the suggested colour scheme (make one plot for each slice). Finally, make a third plot with galaxies from both redshift slices. Add axis labels, a title and an legend to each plot.


In [ ]:
#Solution slice1 (4 points)

In [ ]:
# Solution slice2 (4 points)

In [ ]:
# Solution both slice1 and slice2 (4 points)

### Exercise:

Do you see more structure in the distribution of galaxies in each slice, when compared to your first plot that included all galaxies? 

What can you tell about the structure you see in the two different redshift slices?

Why couldn't you see it in your first plot?

Answer (2 points):


## Galaxy colours

You will see in lectures that the optical colours of galaxies are related to the age of their stars - red galaxies hold older stars, whereas blue galaxies tend to have younger stars. In practice, we can quantify "colour" in Astronomy as _the difference in magnitude in two different bands_.

In this set of exercises **we will focus on the first slice in redshift** (call it sl, which is very narrow, meaning that all galaxies have a similar redshift. I.e., if galaxies in this redshift slice have different colours, it ought to be because their spectra and stellar composition are different, and not because some are redshifted due to the expansion of the Universe. 

Plot a histogram of u-g color (really just column "u" minus column "g") of the galaxies in the slice  0.02 < z < 0.03. Don't forget to label your axes and title your plot!


In [ ]:
# Answer (4 points)

**np.percentile()** (https://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.percentile.html) allows you to quickly return the percentile of a distribution of points. For example, to find the median (50th percentile) u-g colour of your galaxy population you can write:

In [ ]:
median_umg = np.percentile(all_gals.loc[slice1]['u']-all_gals.loc[slice1]['g'], 50)
print(median_umg)

i.e., 50% of the galaxies in your sample have u-g colours that are lower than median_umg (i.e., they are bluer than the median), and 50% have  colours that are larger (i.e., they are redder than the median). If I wanted to choose only the 10% reddest galaxies I could do:

In [ ]:
high_umg = np.percentile(all_gals.loc[slice1]['u']-all_gals.loc[slice1]['g'], 90)

very_red_galaxies = np.where((all_gals['z'] > 0.02) & (all_gals['z'] < 0.03) & 
                                (all_gals['u']-all_gals['g'] > high_umg))

### Exercise:

Use np.percentile() to choose the 25% reddest and 25% bluest galaxies in u-g. Then plot their positions on the sky. Do both types of galaxies trace the large-scale structure in a similar way? What can you say about which galaxies preferentially sit on denser parts of the Universe, and which sit on less dense regions? For this exercise it is recommended that you make two plots (one for the red galaxies, and one for the blue), but that you put them side by side to help you compare.


In [ ]:
# 8 points total for all of this
#Answer 

By now you will have started developing an understanding of how galaxies in general are spacially distributed in the Universe and the shape of the cosmic web, and how galaxies' position on the cosmic web and their environment is related to their colour. Next, we will look at the **shape** of galaxies.

## Galaxy morphology

Galaxy morphology studies the shapes of galaxies. 

Here, we do a more systematic exploration of how galaxy shapes are related to other properties.

The next cell provides a bit of code that selects 16 **random** galaxies from your dataframe, and shows you their optical images. 

In [ ]:
#plot a random subset of 16 galaxies
# set thumbnail parameters
width=200           # image width
height=200          # height
pixelsize=0.396     # image scale
plt.figure(figsize=(15, 15))   # display in a 4x4 grid
subPlotNum = 1

#let's do any pre-selection of the galaxies here.
#In this case, let us only look at the galaxies in a thin redshift slice
my_galaxies = np.where( (all_gals['z'] > 0.02) & (all_gals['z'] < 0.03))[0]

i = 0
nGalaxies = 16 #Total number of galaxies to plot
ind = np.random.randint(0,len(my_galaxies), nGalaxies) #randomly selected rows
count=0
for i in ind:           # iterate through the randomly selected rows in the DataFrame
    count=count+1
    print('Getting image '+str(count)+' of '+str(nGalaxies)+'...')
    if (count == nGalaxies):
        print('Plotting images...')
    # This appears to set the scale of the images it gets
    scale=2*all_gals.loc[i]['petror90_r']/pixelsize/width
    img = SkyServer.getJpegImgCutout(ra=all_gals.loc[my_galaxies[i]]['ra'], dec=all_gals.loc[my_galaxies[i]]['dec'], width=width, height=height, scale=scale,dataRelease='DR14')
    plt.subplot(4,4,subPlotNum) # Figures out where to put the new plot
    subPlotNum += 1
    plt.imshow(img)                               # show images in grid
    plt.title(all_gals.loc[my_galaxies[i]]['z'])  # Adds a title!   

### Exercise:
Compute the fraction of galaxies you'd classify as having disks, and the fraction of galaxies you'd classify as being smooth ellipsoids. If you want to improve your statistics, you can rerun the cell above and you will get 16 different galaxies every time...

Answer: (1 point)

### Exercise:
Now using the code from the example above, do the same thing but taking 16 random galaxies that are **red**, according to your earlier definition of red and blue. Once again, classify the galaxies as disks or ellipticals. **Note, to change the color, you only need to change the line that defines my_galaxies.**  In addition, comment any line of code that doesn't have a comment on it. The comment should explain what that line is for.

In [ ]:
# Answer. 7 points total

Answer: (1 point)

### Exercise 9:

Repeat the above exercise, now with blue galaxies. Repeat your classification exercise.

In [ ]:
#Answer
(7 points just like above)


### Exercise:

From the above exercise, what can you say - if anything - about the relationship between colour and morphology?

Answer: (2 points) 

## Morphology and environment

Finally, we will explore the **relationship between morphology and environment**. It is extraordinarily difficult to write a computer programme that determines whether a galaxy has spiral structure or is smooth. To this day, the human eye does better than the most sophisticated algorithms (the reason behind projects such as Galaxy Zoo - https://www.galaxyzoo.org). We will use a very simple proxy for morphology, which is good enough for our purposes: the sersic index. The sersic index tells us how rapidly the light profile of a galaxy is falling from the center, and essentially classifies galaxies' light profiles as being very centrally concentrated (like ellipticals), or as being flatter (like spirals). 

The cell below repeats our initial SQL query, but now it also returns the sersic profile, called sersic_n, and we are already limiting the redshift range to be between 0.02 and 0.03.

In [ ]:
query="""
SELECT p.objId, p.ra,p.dec,p.petror90_r, p.expAB_r,
    p.dered_u as u, p.dered_g as g, p.dered_r as r, p.dered_i as i, 
    s.z, s.plate, s.mjd, s.fiberid, n.sersic_n
FROM galaxy AS p
   JOIN SpecObj AS s ON s.bestobjid = p.objid
   JOIN nsatlas AS n ON n.mjd = s.mjd and n.fiberID = s.fiberID and n.plate = s.plate
WHERE p.petror90_r > 10
  and p.ra between 100 and 250
  and s.z between 0.02 and 0.03
  and p.g < 17
"""
all_gals = CasJobs.executeQuery(query, "dr14")
print(" Query returns " + str(len(all_gals))+ " galaxies")

Make a histogram of the values of the sersic index in your sample. Label your histogram with axes and a title. Make sure the number of bins is reasonable.

In [ ]:
# Answer  (4 points)

### Exercise:

Look at the morphologies of high and low sersic index galaxies, like you did for blue and red. In other words, make a set of 16 plots of high sersic index galaxies, and then a set of 16 plots of low sersic index galaxies. Let us test that idea that galaxies with a very high sersic index (n > 4) are ellipticals , and those with a very low sersic index (n < 2) are spirals. After you've looked at both those sets of plots (you should show them all below) answer this question: In your opinion, how good is this number at identifying morphology?

In [ ]:
# Answer (to plotting the ellipticals, 4 points)


In [ ]:
# Answer (to plotting spirals)
#plot a random subset of 16 SPIRALS (4 points)

Answer: (1 point)  

Write a summary (4-6 sentences) of the gains you made in your astronomical knowledge doing this lab. 

Answer: (3 points) 

Write a summary (4-6 sentences) of the gains you made in your programming skills while doing this lab.

Answer: (3 points) 

**Congratulations, that is the end of the Lab!** Make sure you've **run all the code cells, filled in all the text answers and that your plots are all showing without error**. 